In [16]:
import psycopg2

import random
import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

import joblib

import torch
from torch import nn
from torch.utils.data import Subset, DataLoader

from Dataset.Embedding_Dataset import Embedding_Dataset
from Model.Embedding import Embedding

from Dataset.Apartment_Complex_Dataset import Apartment_Complex_Dataset
from Model.LSTM import LSTM
from Model.GRU import GRU
from Model.NLinear import NLinear
from Model.Transformer import Transformer2

from Dataset.Dong_Dataset import Dong_Dataset
from Model.LSTM_Attention_E2E import LSTMSeq2Seq
from Model.LSTM_Attention import LSTMAttention
from Model.GRU_Attention import GRUAttention
from Model.Transformer_Attention_E2E import TransformerSeq2Seq
from Model.Transformer_Attention import TransformerAttention
from Model.NLinear_Attention import NLinearAttention

from utils import *

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = torch.device('cpu')

## Hyperparameters

In [17]:
embedding_dim = 1024
window_size = 10
batch_size = 1

ml_batch = 128
ml_estimators = 150
ml_window_size = 10

## Data

In [18]:
table_1 = pd.read_csv('../데이터/Table/test_table_1.csv') 
table_2 = pd.read_csv('../데이터/Table/test_table_2.csv') 
table_3 = pd.read_csv('../데이터/Table/test_table_3.csv') 

## DL Dataset & Dataloader

In [19]:
# concat_12_dim
# dataset = Dong_Dataset('None', table_1, table_2, table_3, 'None', window_size, 'TEST', DEVICE)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# emb_1024_dim
model = torch.load('../데이터/Checkpoint/emb_1024_dim/embedding_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_131_e1_128_e2_128_e3_512_emb_1024_d1_512_d2_256_d3_128.pth', map_location=torch.device('cpu'))
dataset = Dong_Dataset(model, table_1, table_2, table_3, embedding_dim, window_size, 'TEST', DEVICE)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [26]:
# LSTM
# model = torch.load("../데이터/Checkpoint/concat_12_dim/lstm_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_124_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/lstm_tr_0.8_lr_1e-06_wd_0_batch_128_epochs_134_hdim_256_ws_10.pth", map_location=torch.device('cpu'))

# GRU
# model = torch.load("../데이터/Checkpoint/concat_12_dim/gru_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_321_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/gru_tr_0.8_lr_1e-06_wd_0_batch_128_epochs_164_hdim_256_ws_10.pth", map_location=torch.device('cpu'))

# nlinear
# model = torch.load("../데이터/Checkpoint/concat_12_dim/nlinear_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_118_emb_12_ws_10.pth", map_location=torch.device('cpu'))

# transformer
# model = torch.load("../데이터/Checkpoint/concat_12_dim/transformer_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_63_ws_10.pth", map_location=torch.device('cpu'))
model = torch.load("../데이터/Checkpoint/transformer_tr_0.8_lr_1e-06_wd_0_batch_1_epochs_9_ws_10.pth", map_location=torch.device('cpu'))

# LSTM attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/lstm_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/lstm_att_tr_0.8_lr_1e-06_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))

# GRU attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/gru_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/gru_att_tr_0.8_lr_1e-06_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))

# NLinear attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/nlinear_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_6.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/emb_1024_dim/", map_location=torch.device('cpu'))

# Transformer attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/transformer_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_7_hdim_12_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/transformer_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_15_hdim_1024_ws_10.pth", map_location=torch.device('cpu'))

In [27]:
RMSE_criterion = RMSE()
MAE_criterion = MAE()

count = 0
RMSE_total_loss = 0
MAE_total_loss = 0

with torch.no_grad():
    for i, batch in enumerate(dataloader):
        src = batch[0][0].to(DEVICE)
        max_len = batch[1][0].to(DEVICE)
        anw = batch[2][0]
        trg = batch[3][0].to(DEVICE)
        
        if len(anw) == 0:
            continue
        
        RMSE_epoch_loss = 0
        MAE_epoch_loss = 0
        count += anw.shape[0]

        for index in anw:
            index.to(DEVICE)
            # LSTM
            # output, _, _ = model(src)
            
            # GRU
            # output, _ = model(src)
            
            # nlinear
            # output, _ = model(src)
            
            # transformer
            src_mask = model.generate_square_subsequent_mask(src.shape[1]).to(src.device)
            output, _ = model(src, src_mask)
            
            RMSE_loss = RMSE_criterion(output[index], trg[index])
            MAE_loss = MAE_criterion(output[index], trg[index])
            
            # attention 계열
            # output = model(src, index, max_len)
            # RMSE_loss = RMSE_criterion(output, trg[index])
            # MAE_loss = MAE_criterion(output, trg[index])
            
        RMSE_total_loss += RMSE_loss
        MAE_total_loss += MAE_loss

    print(f'Test RMSE Loss: {RMSE_total_loss/count:.4f}')
    print(f'Test MAE Loss: {MAE_total_loss/count:.4f}')

Test RMSE Loss: 2.5371
Test MAE Loss: 2.5371


## ML Dataset & Dataloader

In [ ]:
# concat_12_dim
dataset = Apartment_Complex_Dataset('None', table_1, table_2, table_3, 'None', ml_window_size, 'ML', 'TEST', DEVICE)
dataloader = DataLoader(dataset, batch_size=ml_batch, shuffle=False, drop_last=True)

# emb_1024_dim
# model = torch.load('../데이터/Checkpoint/embedding_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_131_e1_128_e2_128_e3_512_emb_1024_d1_512_d2_256_d3_128.pth', map_location=torch.device('cpu'))
# dataset = Apartment_Complex_Dataset(model, table_1, table_2, table_3, embedding_dim, ml_window_size, 'ML', 'TEST', DEVICE)
# dataloader = DataLoader(dataset, batch_size=ml_batch, shuffle=False, drop_last=True)

In [ ]:
# lightgbm
# model =  joblib.load('../데이터/Checkpoint/concat_12_dim/lightgbm_tr_0.8_batch_128_estimators_150_ws_10.pkl')
# catboost
model =  joblib.load('../데이터/Checkpoint/concat_12_dim/catboost_tr_0.8_batch_128_estimators_150_ws_10.pkl')

RMSE_total_loss = 0
MAE_total_loss = 0

for i, data in enumerate(dataloader):
    X, y = data[0].squeeze().cpu().numpy(), data[1].squeeze().cpu().numpy()
    y_pred = model.predict(X)
    RMSE_loss = MSE(y, y_pred)
    RMSE_total_loss += RMSE_loss/len(dataloader)
    MAE_loss = MAE(y, y_pred)
    MAE_total_loss += MAE_loss/len(dataloader)

print(f'Test RMSE Loss: {RMSE_total_loss:.4f}')
print(f'Test MAE Loss: {MAE_total_loss:.4f}')